In [1]:
cd ..

/Users/nofar/Dropbox (Weizmann Institute)/Nofar Azulay’s files/Home/Code/test/mibi-bin-tools


In [2]:
# import required packages
import os
import json
import mmap
import multiprocessing as mp
import numpy as np
import pandas as pd
import skimage.io as io
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import numpy as np

from mibi_bin_tools import bin_files, io_utils

plt.rcParams["figure.figsize"] = (20,13)
plt.rcParams["ytick.color"] = "w"
plt.rcParams["xtick.color"] = "w"
plt.rcParams["axes.labelcolor"] = "black"
plt.rcParams["axes.edgecolor"] = "w"
plt.rcParams["axes.facecolor"] = 'black'
plt.rcParams["savefig.edgecolor"] = "w"
plt.rcParams["savefig.facecolor"] = 'black'
plt.rcParams["figure.facecolor"] = 'black'
plt.rcParams["figure.constrained_layout.use"] = False
plt.rcParams["legend.facecolor"] = 'w'

### Add a folder to '/data' with bin and jason files for all fovs

In [3]:
# set up file paths
base_dir = "data/hackathon_bins"
data_dir = base_dir

extracted_dir = os.path.join(base_dir, 'extracted')

In [4]:
# create directories if do not exist
for directory in [extracted_dir]:
    if not os.path.exists(directory):
        os.makedirs(directory)

# Plot spectra - raw and calibrated

### You can skip this step and go to extraction if you want to use automatic calibration

In [5]:
#select fov (only one) to extract spectrum
include_fovs = ['Duodneum_B3']

In [6]:
#extracts and saves spectrum.csv to extracted/spectrum folder
spectra_data = bin_files.extract_spectra(data_dir, extracted_dir, include_fovs=include_fovs)

### Plot spectrum before calibration

In [14]:
%matplotlib tk

masses_list = np.arange(23,200)
fig, axs = plt.subplots(1, 2 , constrained_layout=True )
axs[0].plot(spectra_data['TimeOffset'] , spectra_data['Counts'] ,'g' , label = 'Spectrum in time')
axs[0].legend(loc="upper right")
axs[0].set_ylabel('Counts')
axs[0].set_xlabel('Time [A/D units]')
axs[1].plot(spectra_data['massList'] ,spectra_data['Counts'] , label = 'Spectrum in mass (auto calibration)')
axs[1].set_ylabel('Counts')
axs[1].set_xlabel('Mass [dalton]')
axs[1].legend(loc="upper right")
axs[1].vlines(x = masses_list, ymin = 0 , ymax = np.max(spectra_data['Counts'])/2 ,color = 'red', lw=0.5)

### Insert values based on spectra (m1,t1) , (m2, t2)

In [15]:
m1 = 22.98976928
m2 = 196.966
t1 = 1041
t2 = 13292

In [16]:
#get calibration parameters
mass_offset , mass_gain = bin_files.calibrate_spectrum(t1 , m1 , t2 , m2)
spectra_data['mass_cal'] = pd.Series(bin_files.tof2mass(spectra_data['TimeOffset'].to_numpy(), mass_offset, mass_gain))

### Plot new vs. old calibration

In [17]:
%matplotlib tk

fig, axs = plt.subplots(1, 2 , constrained_layout=True , sharex=True , sharey=True)
axs[0].plot(spectra_data['mass_cal'] , spectra_data['Counts'] ,'g' , label = 'Spectrum in mass (maual calibration)')
axs[0].set_title('New calibration')
axs[0].set_ylabel('Counts')
axs[0].set_xlabel('Time [A/D units]')
axs[0].legend(loc="upper right")
axs[0].vlines(x = masses_list, ymin = 0 , ymax = np.max(spectra_data['Counts'])/2 ,color = 'red', lw=0.5)
axs[1].plot(spectra_data['massList'] ,spectra_data['Counts'] , label = 'Spectrum in mass (auto calibration)')
axs[1].set_ylabel('Counts')
axs[1].set_xlabel('Mass [dalton]')
axs[1].set_title('current calibration')
axs[1].legend(loc="upper right")
axs[1].vlines(x = masses_list, ymin = 0 , ymax = np.max(spectra_data['Counts'])/2 ,color = 'red', lw=0.5)

# Extract bin files

#include_foves - list of fovs to extract, if 'None' all bin files in folder are extracted.

#panel - tuple of integration window or csv file with 'Start' and 'Stop' columns.

#calibration - tuple of (mass offset, mass_gain) , if 'auto' using the machine parameters.

In [28]:
#include_fovs = ['Duodneum_B3']
include_fovs = None

panel = (-0.3, 0)
#panel = pd.read_csv(os.path.join(base_dir,'panel.csv'))

#calibration = 'auto'
calibration = (mass_offset , mass_gain)

In [29]:
bin_files.extract_bin_files(data_dir, extracted_dir, include_fovs=include_fovs, panel=panel , calibration = calibration)